<a href="https://colab.research.google.com/github/fpgmina/DeepNLP/blob/main/HuggingFace_TimesFM_example_usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

HuggingFace: https://huggingface.co/google/timesfm-2.5-200m-pytorch

GitHub: https://github.com/google-research/timesfm

In [ ]:
# 1. Install the timesfm library (Run this in a Colab cell)
!pip install git+https://github.com/google-research/timesfm.git

In [ ]:
import numpy as np
import torch
import timesfm
import matplotlib.pyplot as plt

In [ ]:
# 3. Load and Compile the Model (This may take a moment)
torch.set_float32_matmul_precision("high")
model = timesfm.TimesFM_2p5_200M_torch.from_pretrained("google/timesfm-2.5-200m-pytorch")

# Define model configuration (use the standard defaults for a quick start)
model.compile(
    timesfm.ForecastConfig(
        max_context=1024,
        max_horizon=256,
        normalize_inputs=True,
    )
)

In [ ]:
# 4. Prepare Example Input Data
# Use a numpy array of historical time series values
# The model is univariate (one series at a time)
historical_data_1 = np.linspace(0, 10, 100)  # Example 1: 100 context points
historical_data_2 = np.sin(np.linspace(0, 20, 67)) # Example 2: 67 context points

# 5. Make a Forecast
horizon_steps = 12
point_forecast, quantile_forecast = model.forecast(
    horizon=horizon_steps,
    inputs=[historical_data_1, historical_data_2] # Pass your series as a list
)

# Output shapes:
# point_forecast.shape -> (2, 12)  # (Number of Series, Horizon)
# quantile_forecast.shape -> (2, 12, 10) # (Series, Horizon, Quantiles)

In [ ]:
point_forecast

## Example Usage with Sine Function

In [ ]:
# --- 1. Generate the Full Sine Wave Data ---
# A common setup: 4 full cycles (e.g., 400 data points)
n_cycles = 4
n_points_per_cycle = 100
n_total = n_cycles * n_points_per_cycle

# Generate the true underlying sine series
t = np.linspace(0, n_cycles * 2 * np.pi, n_total, endpoint=False)
full_series = 10 * np.sin(t)

# --- 2. Define Context (Input) and True Horizon (Ground Truth) ---
context_len = 300  # Use the first 3 cycles/300 points as input
horizon_len = 100  # Predict the last cycle/100 points

historical_data = full_series[:context_len]
true_future = full_series[context_len:context_len + horizon_len]

# Note: TimesFM is designed for a minimum context of ~30 points.
# Using a large context (300) gives it ample pattern data.

In [ ]:
# (Assuming TimesFM library is installed and the model object 'model' is loaded and compiled)

# Generate the forecast (TimesFM expects inputs as a list)
point_forecast, _ = model.forecast(
    horizon=horizon_len,
    inputs=[historical_data]
)

# Extract the single point forecast array
forecast_values = point_forecast[0]

In [ ]:
mae = np.mean(np.abs(true_future - forecast_values))
rmse = np.sqrt(np.mean((true_future - forecast_values)**2))

print(f"\nMean Absolute Error (MAE): {mae:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

In [ ]:
plt.figure(figsize=(12, 6))

# Plot Historical Data
plt.plot(range(context_len), historical_data, label='Historical Input', color='blue')

# Plot True Future
future_range = range(context_len, context_len + horizon_len)
plt.plot(future_range, true_future, label='True Future (Ground Truth)', color='green', linestyle='--')

# Plot Forecast
plt.plot(future_range, forecast_values, label='TimesFM Forecast', color='red', linestyle='-', alpha=0.7)

plt.axvline(x=context_len, color='gray', linestyle=':', label='Forecast Start')
plt.title('TimesFM Forecast on Sine Time Series')
plt.xlabel('Time Step')
plt.ylabel('Value')
plt.legend()
plt.grid(True, which='both', linestyle='--')
plt.show()